# Procesamiento de datos

In [2]:
import os

import librosa
from librosa import display

import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import IPython
import IPython.display as ipd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

### variables de entorno

In [3]:
DATASET = './DataSets/Emotions'
LABELS = ['Angry', 'Happy', 'Sad', 'Fearful', 'Disgusted', 'Neutral', 'Suprised']
SAMPLE_RATE = 22050

### funciones de procesamiento

In [4]:
def remove_silence(signal, threshold=0.005):
    """
    Remove silence at the beginning and at the end of the signal
    """
    for i in range(len(signal)):
        if np.abs(signal[i]) > threshold:
            break
    for j in range(len(signal)-1, 0, -1):
        if np.abs(signal[j]) > threshold:
            break
    return signal[i:j]

In [52]:
resize = int(SAMPLE_RATE*2)
print(resize)
def signal_resize(signal, length=resize):
    """
    Cut the signal to the given length, or pad it with zeros if it is shorter
    """
    if len(signal) > length:
        return signal[:length]
    else:
        return np.pad(signal, (0, max(0, length - len(signal))), "constant")

44100


### Read the dataset

In [55]:
emotions = []

for dirname, dirs, filenames in os.walk(DATASET):
    print(dirname)
    for filename in filenames[:10]:
        path = os.path.join(dirname, filename)
        title = dirname.split('\\')[-1]
        category = LABELS.index(title)

        signal, sample_rate = librosa.load(path, sr=SAMPLE_RATE)
        signal = remove_silence(signal)
        mean = np.mean(signal)
        std = np.std(signal) # Standard deviation

        signal = signal_resize(signal)
        time = len(signal) / sample_rate
        zero_crossings = sum(librosa.zero_crossings(signal, pad=False))
        zero_crossing_rate = librosa.feature.zero_crossing_rate(signal)[0]
        stft_mag, stft_phase = librosa.magphase(librosa.stft(signal, n_fft=512)) # Short-time Fourier transform, n_fft = 512 for 22050 Hz
        freqs = librosa.fft_frequencies(sr=sample_rate)
        max_stft_freq = freqs[int(np.argmax(stft_mag)/stft_mag.shape[1])]
        min_stft_freq = freqs[int(np.argmin(stft_mag)/stft_mag.shape[1])]
        stft_phase = np.angle(stft_phase)
        rms = librosa.feature.rms(S=stft_mag)[0] # Root mean square - Energy
        mfcc = librosa.feature.mfcc(S=stft_mag, sr=sample_rate) # Mel-frequency cepstral coefficients

        emotions.append((
            path,
            title,
            category, 
            signal, 
            mean, 
            std, 
            sample_rate, 
            time, 
            zero_crossings, 
            zero_crossing_rate, 
            stft_mag, 
            stft_phase, 
            max_stft_freq,
            min_stft_freq,
            rms,
            mfcc,
        ))

./DataSets/Emotions
./DataSets/Emotions\Angry
./DataSets/Emotions\Disgusted
./DataSets/Emotions\Fearful
./DataSets/Emotions\Happy
./DataSets/Emotions\Neutral
./DataSets/Emotions\Sad
./DataSets/Emotions\Suprised


In [34]:
columns = [
    'path',
    'title',
    'category', 
    'signal', 
    'mean',
    'std',
    'sample_rate', 
    'time', 
    'zero_crossings', 
    'zero_crossing_rate',
    'stft_mag',
    'stft_phase',
    'max_stft_freq',
    'min_stft_freq',
    'rms',
    'mfcc',
]

In [56]:
emotions_frame = pd.DataFrame(emotions, columns=columns)
emotions_frame.head()

,path,title,category,signal,mean,std,sample_rate,time,zero_crossings,zero_crossing_rate,stft_mag,stft_phase,max_stft_freq,min_stft_freq,rms,mfcc
0,./DataSets/Emotions\Angry\03-01-05-01-01-01-01...,Angry,0,"[0.0057457346, 0.0022356973, -0.005082519, -0....",3.367120e-05,0.021871,22050,1.831247,5117,"[0.173828125, 0.2548828125, 0.3017578125, 0.22...","[[0.010907069, 0.04286768, 0.043306984, 0.1027...","[[0.0, 3.1415927, 3.1415927, 0.0, 0.0, 3.14159...",742.895508,11025.000000,"[0.0030020439008907264, 0.004094262346243608, ...","[[3.023506, 4.1251388, 3.6748028, 4.0287776, 6..."
1,./DataSets/Emotions\Angry\03-01-05-01-01-01-02...,Angry,0,"[0.008221892, 0.010217116, -0.004288578, -0.00...",-6.661139e-07,0.018163,22050,1.624036,4107,"[0.19482421875, 0.29345703125, 0.37060546875, ...","[[0.0026830183, 0.004631814, 0.0034631253, 0.0...","[[0.0, 3.1415927, 0.0, 0.0, 0.0, 3.1415927, 3....",1044.360352,10928.100586,"[0.002784303083899896, 0.004123150881421557, 0...","[[2.8428097, 4.1928973, 3.7210562, 3.3208735, ..."
2,./DataSets/Emotions\Angry\03-01-05-01-01-01-03...,Angry,0,"[0.005050677, 0.004881932, 0.0029387863, -0.00...",-7.153223e-07,0.005678,22050,1.986984,5028,"[0.18017578125, 0.2685546875, 0.34716796875, 0...","[[0.0043746997, 0.0034669111, 0.0019069077, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",409.130859,11025.000000,"[0.0006149001893383993, 0.0012496158455687625,...","[[0.6603924, 1.3115329, 1.1970788, 0.92137367,..."
3,./DataSets/Emotions\Angry\03-01-05-01-01-01-04...,Angry,0,"[0.007672126, 0.009416925, 0.0026265364, -0.00...",-3.689617e-07,0.021310,22050,1.657959,4449,"[0.17333984375, 0.248046875, 0.26416015625, 0....","[[0.002413682, 0.00275122, 0.0014988944, 0.004...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.1415927...",1205.859375,10981.933594,"[0.001554837728977314, 0.0022501271001912925, ...","[[1.5061591, 2.1449115, 2.1883314, 4.1438575, ..."
4,./DataSets/Emotions\Angry\03-01-05-01-01-01-05...,Angry,0,"[0.0069410847, 0.015283292, 0.0083997715, -0.0...",-7.673296e-06,0.025376,22050,1.911474,4986,"[0.1826171875, 0.240234375, 0.263671875, 0.218...","[[0.6866143, 1.0526644, 0.46180674, 2.522717, ...","[[0.0, 0.0, 3.1415927, 3.1415927, 3.1415927, 0...",689.062500,11025.000000,"[0.0031879409585994427, 0.00516772450047762, 0...","[[3.3411024, 5.700085, 8.216389, 13.678373, 20..."
